In [1]:
import pathlib
import argparse
import sys
import os

sys.path.append("/workspace/rs-data-python")
sys.path.append(".")

import pandas as pd
import numpy as np

from data_utils import init_random
init_random()

In [2]:
from data_groups import GroupDataFT
from data_groups import OneHotGenerator
dataset = GroupDataFT()
ds_shape, nuser, nitems, ds_code = dataset.get_shape(), dataset.get_num_users(), dataset.get_num_items(), dataset.get_data_code()

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Embedding, Input, Dense, Reshape, Flatten, Concatenate, Multiply
import numpy as np
import matplotlib.pyplot as plt

k=8
seed=37
model_name='preliminar'

from keras.regularizers import l2

regs=[0,0]
    
input_layer = layers.Input(shape=ds_shape, name="entrada")

e_user = layers.Dense(
    k,
    name="emb_u",
    kernel_initializer = 'normal', kernel_regularizer = l2(regs[0])
)(layers.Lambda(lambda x: x[:, 0:nuser])(input_layer))

e_item = layers.Dense(
    k,
    name="emb_i",
    kernel_initializer = 'normal', kernel_regularizer = l2(regs[1])
)(layers.Lambda(lambda x: x[:, nuser:])(input_layer))

# Crucial to flatten an embedding vector!
user_latent = Flatten()(e_user)
item_latent = Flatten()(e_item)

# Element-wise product of user and item embeddings 
#predict_vector = merge([user_latent, item_latent], mode = 'mul')
vector = Concatenate()([user_latent, item_latent])
# MLP layers
for idx in [64,32,16,8]:
    layer = Dense(idx, kernel_regularizer= l2(0), activation='relu', name = 'layer%d' %idx)
    vector = layer(vector)

# Final prediction layer
#prediction = Lambda(lambda x: K.sigmoid(K.sum(x)), output_shape=(1,))(predict_vector)
#prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(predict_vector)
# CHANGED to linear for regression
outputs = Dense(1, activation='linear', kernel_initializer='lecun_uniform', name = 'prediction')(vector)

model = keras.Model(inputs=input_layer, outputs=outputs, name=model_name)
model.summary()

Model: "preliminar"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
entrada (InputLayer)            [(None, 3579)]       0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1508)         0           entrada[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 2071)         0           entrada[0][0]                    
__________________________________________________________________________________________________
emb_u (Dense)                   (None, 8)            12072       lambda[0][0]                     
_________________________________________________________________________________________

In [4]:
BATCH=64
EPOCH=25

num_users = dataset.get_num_users()
num_items = dataset.get_num_items()
(x_train, x_val, y_train, y_val) = dataset.get_train_val()
train_secuencer = OneHotGenerator(x_train, y_train, num_users, num_items, BATCH)
val_secuencer = OneHotGenerator(x_val, y_val, num_users, num_items, BATCH)

model.summary()
model.compile(
    loss=tf.keras.losses.MeanAbsoluteError(),
    optimizer=keras.optimizers.Adam(lr=0.001)
    #optimizer=keras.optimizers.Nadam()
)

history = model.fit(
    train_secuencer,
    validation_data=val_secuencer,
    epochs=EPOCH,
    verbose=1
)



Model: "preliminar"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
entrada (InputLayer)            [(None, 3579)]       0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1508)         0           entrada[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 2071)         0           entrada[0][0]                    
__________________________________________________________________________________________________
emb_u (Dense)                   (None, 8)            12072       lambda[0][0]                     
_________________________________________________________________________________________

In [5]:
(x_test, y_test) = dataset.get_test()
test_secuencer = OneHotGenerator(x_test, y_test, num_users, num_items, BATCH)

results = model.evaluate(test_secuencer)

45/45 [==============================] - 0s 1ms/step - loss: 0.6592


In [6]:
results

0.6592480540275574

In [13]:
input_layer = layers.Input(shape=ds_shape, name="entrada")

e_user = layers.Dense(
    k,
    name="emb_u",
    kernel_initializer = 'normal', kernel_regularizer = l2(regs[0])
)(layers.Lambda(lambda x: x[:, 0:nuser])(input_layer))

user_latent = Flatten()(e_user)

vector = user_latent
# MLP layers
for idx in [64,32,16,nuser]:
    layer = Dense(idx, kernel_regularizer= l2(0), activation='relu', name = 'layer%d' %idx)
    vector = layer(vector)

one_hot_item = layers.Lambda(lambda x: x[:, nuser:])(input_layer)
multihot_user = vector
mlp_agg = Concatenate()([multihot_user, one_hot_item])
model.trainable = False

agg_mlp_model = model(mlp_agg)

model = keras.Model(inputs=input_layer, outputs=agg_mlp_model, name='mlp_agg')
model.summary()

Model: "mlp_agg"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
entrada (InputLayer)            [(None, 3579)]       0                                            
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 1508)         0           entrada[0][0]                    
__________________________________________________________________________________________________
emb_u (Dense)                   (None, 8)            12072       lambda_8[0][0]                   
__________________________________________________________________________________________________
flatten_6 (Flatten)             (None, 8)            0           emb_u[0][0]                      
____________________________________________________________________________________________

In [12]:
mlp_agg.shape

TensorShape([None, 3579])